### installation guide
First I needed the postgresql database from musicbrainz. It turned out the documentation for setting up the server was quite a bit behind. However, after some trial and error, I found a good way to set up the database:

follow [the musicbrainz documentation](https://musicbrainz.org/doc/MusicBrainz_Server/Setup) until accessing the database.

To get access to the virtual machine, log in (or ssh `ssh -p 2222 vagrant@localhost`) with username `vagrant` and password `vagrant`.

Docker is not configured to expose the database by default. To change the configuration, diff the following lines to `musicbrainz/musicbrainz-docker/docker-compose.yml`

```
14,15d13
<     ports:
<       - "5432:5432"
20a19,20
>     expose:
>       - "5432"
```

Now, in virtualbox, port forward (settings -> network -> advanced -> port forwarding) port 5432. This enables you to connect to the postgres database from your host machine.

After this, you can connect with user `musicbrainz` with password `musicbrainz` on your host machine (for example with `psql -h localhost -U musicbrainz -p 5432`)

In [1]:
import psycopg2

conn = psycopg2.connect(user="musicbrainz", password="musicbrainz")

One of the main early objectives of this project is to get all the aliases for a given composer (or other type of artist, since musicbrainz does not destinguish between the two).

The function below shows a simple implementation of this.

In [2]:
def get_aliases(person):
    '''
    Gets all the aliases for a given person by querying both the artist_credit_name
    table and the artist_alias table. The function throws away duplicates by returning
    the result as a set.
    '''
    with conn:
        with conn.cursor() as curs:
            curs.execute(
                """
                select artist_credit_name.name 
                from artist
                left join artist_credit_name on artist_credit_name.artist = artist.id 
                where artist.name = %s
                """, [person]
            )
            credit_names = curs.fetchall()
            credit_names = {name[0] for name in credit_names}

            curs.execute(
                """
                select artist_alias.name 
                from artist
                left join artist_alias on artist.id = artist_alias.artist 
                where artist.name = %s
                """, [person]
            )

            aliases = curs.fetchall()
            aliases = {name[0] for name in aliases}

            return credit_names | aliases

get_aliases('Alexander von Zemlinsky')   
    

{'Alexander Zemlinsky',
 'Alexander von Zemlinsky',
 'Zemlinksy',
 'Zemlinsky',
 'ツェムリンスキー'}

As you can see, the list of aliases contains multiple different spellings of Zemlinsky.



In [3]:
get_aliases('Wolfgang Amadeus Mozart')

{'English Classical Players',
 'Johannes Chrysostomus Wolfgangus Theophilus Mozart',
 'Mosart',
 'Mozart',
 'Volfgangs Amadejs Mocarts',
 'W A Mozart',
 'W. A. Mozart',
 'W. A. Mozart/Arr. by Nova',
 'W. Amadeus Mozart',
 'W.-A. Mozart',
 'W.A Mozart',
 'W.A. Mozart',
 'W.A.Mozart',
 'WA Mozart',
 'WA. Mozart',
 'Wolfang A. Mozart',
 'Wolfang Amadeus Mozart',
 'Wolfgan Amadeus Mozart',
 'Wolfgang A. Mozart',
 'Wolfgang Amade Mozart',
 'Wolfgang Amadeaus Mozart',
 'Wolfgang Amadeus Mozart',
 'Wolfgang Amadeus Mozart (1756-1791)',
 'Wolfgang Amadues Mozart',
 'Wolfgang Amedeus Mozart',
 'Wolfgang Anadeus Mozart',
 'Wolfgang Armadeus Mozart',
 'Wolfgang Mozart',
 'mozart',
 'Вольфганг Амадей Моцарт',
 'Моцарт',
 'モーツァルト',
 'ヴォルフガンク・アマデウス・モーツァルト',
 '볼프강 아마데우스 모짜르트'}

Using a more well known composer as input gives even more useful results.

### Creating a searchable database from these queries
De volgende stap is het verzamelen van deze data voor alle artiesten in de database en die vervolgens doorzoekbaar te maken. Cynthia stelde voor om de aliases om te zetten naar n-grams (als ik het goed heb begrepen). Deze resultaten zijn over het algemeen echter zo kort dat het mij persoonlijk beter lijkt om eerst een simpelere aanpak te gebruiken en de aliases allemaal bij elkaar te halen en vervolgens te tokenizen (op spaties, punten en een aantal andere leestekens, zoals '/'). Vervolgens zijn er voor iedere persoon dus een aantal woorden die daarmee corresponderen. Die woorden worden de zoektermen in de database. Oftewel, een [bag of words met Tf-idf (term frequency, inverse document frequency)](https://nlp.stanford.edu/IR-book/html/htmledition/term-frequency-and-weighting-1.html). 

Vervolgens kan de artiesten zoek database makkelijk worden uitgebreid met de lijst van werken en de lijst van cd's in de database. Dezelfde bag of words kan namelijk ook worden toegepast op cd's en werken. Waar iedere afzonderlijke bag een lijst is met de naam en alias van de componist, de uitvoerders en alle relevante informatie van het werk en de cd zelf. Ik verwacht dat je daarmee al heel veel relevante data kan krijgen. Het is dan zelfs mogelijk om bijvoorbeeld de metadata van de componist zwaarder te laten wegen dan de metadata van de cd of omgekeerd via [weighted zone scoring](https://nlp.stanford.edu/IR-book/html/htmledition/weighted-zone-scoring-1.html). 

De volgende stap is spelling correctie op de individuele termen. Als dit is geimplementeerd en de resultaten nog niet relevant genoeg zijn kunnen we ook nog de positie van de zoektermen meenemen via n-grams.

Ik hoop dat je dit plan van aanpak een goed idee vindt!

The Artist database is a very useful resource for composer search. However, I would also like to search for individual works. the work database seems to be the most useful for this purpose (other contenders being Releases and Recordings). Just like the artist database, a useful table is the "alias" table. Besides that, two other tables are relevant, l_artist_to_work (which relates artists to works) and l_work_to_recording

In [4]:
def get_works(person, with_type=False):
    with conn:
        with conn.cursor() as curs:
            if with_type:
                curs.execute(
                    """
                    select work.name, work_type.name, artist.name from work 
                    left join l_artist_work on entity1=work.id 
                    left join artist on entity0=artist.id 
                    left join work_type on work.type=work_type.id 
                    where artist.name = %s;
                    """, [person]
                )
            else:
                curs.execute(
                    """
                    select work.id, work.name, artist.name from work 
                    left join l_artist_work on entity1=work.id 
                    left join artist on entity0=artist.id 
                    where artist.name = %s;
                    """, [person]
                )
            works = curs.fetchall()
            
            return works
        
        
get_works('Robert Muczynski')[1:25]

[(12834141,
  'Concerto no. 1 for Piano and Orchestra, op. 7: II. Allegretto Pastorale',
  'Robert Muczynski'),
 (12834142,
  'Concerto no. 1 for Piano and Orchestra, op. 7: III. Vivace',
  'Robert Muczynski'),
 (12834143, 'A Serenade for Summer, op. 38', 'Robert Muczynski'),
 (12834144, 'Suite, op. 13: I. Festival', 'Robert Muczynski'),
 (12834145, 'Suite, op. 13: II. Flight', 'Robert Muczynski'),
 (12834146, 'Suite, op. 13: III. Vision', 'Robert Muczynski'),
 (12834147, 'Suite, op. 13: IV. Labyrinth', 'Robert Muczynski'),
 (12834148, 'Suite, op. 13: V. Phantom', 'Robert Muczynski'),
 (12834149, 'Suite, op. 13: VI. Scherzo', 'Robert Muczynski'),
 (12834150,
  'Concerto no. 1 for Piano and Orchestra, op. 7',
  'Robert Muczynski'),
 (12834151, 'Suite, op. 13', 'Robert Muczynski'),
 (12834152,
  'Quintet for Winds, op. 45: I. Allegro risoluto',
  'Robert Muczynski'),
 (12834153, 'Quintet for Winds, op. 45: II. Andante', 'Robert Muczynski'),
 (12834154,
  'Quintet for Winds, op. 45: III. 

In [5]:
get_works("City of Prague Philharmonic")

[]

In [6]:
def get_work_names(work_id):
    with conn:
        with conn.cursor() as curs:
            curs.execute(
                """
                select work.name from work
                where work.id=%s
                """, [work_id]
            )
            name = curs.fetchall()[0][0]
            
            curs.execute(
                """
                select work_alias.name from work_alias 
                left join work on work.id=work_alias.work 
                where work.id=%s;
                """, [work_id]
            )
            
            names = {name[0] for name in curs.fetchall()}
            names.add(name)
            
    return names
        
get_work_names(357993)
    

{'Cello Suite No. 1 in G major, BWV 1007: Gigue',
 'Cello Suite No. 1 in G major, BWV 1007: VI. Gigue',
 'Suite No. 1 for Violoncello Solo in G major, BWV 1007: VII. Gigue',
 'Suite für Violoncello solo no. 1 G-Dur, BWV 1007: VI. Gigue'}

These aliases seem most useful in an "unstructured document approach".

In [7]:
def get_recordings(work_id):
    with conn:
        with conn.cursor() as curs:
            curs.execute(
                """
                select artist_credit.name, recording.name from recording 
                left join l_recording_work on entity0=recording.id 
                left join work on entity1=work.id 
                left join artist_credit on artist_credit.id=artist_credit 
                where work.id=%s;
                """, [work_id]
            )
            return {artist[0] for artist in curs.fetchall()}

        
get_recordings(3566296)

{'Alexander Kniazev',
 'Alexander Rudin',
 'Ana Vidović',
 'André Navarra',
 'Anne Gastinel',
 'Anner Bylsma',
 'Antonio Janigro',
 'Antônio Meneses',
 'Barbara Westphal',
 'Boris Pergamenschikow',
 'Béatrice Reibel',
 'Catherine Toulouse-Delpeuch',
 'Csaba Onczay',
 'Daniel Domb',
 'Daniel Yeadon',
 'David Geringas',
 'David Watkin',
 'Edgar Meyer',
 'Erkki Rautio',
 'Erling Blöndal Bengtsson',
 'Evžen Rattay',
 'Frédéric Dupuis',
 'Gaspar Cassadó',
 'Gavriel Lipkind',
 'Guido Schiefen',
 'Guy Danel',
 'Heinrich Schiff',
 'Hidemi Suzuki',
 'Jaap ter Linden',
 'Jacqueline du Pré',
 'Jean‐Guihen Queyras',
 'Jian Wang',
 'Justin Pearson',
 'János Starker',
 'Jörg Baumann',
 'Jörg Metzger, City of Prague Philharmonic Orchestra, James Fitzpatrick',
 'Klaus-Peter Hahn',
 'Laurie Haines Reese',
 'Lillian Fuchs',
 'Lluís Claret',
 'Luigi Piovano',
 'Lynn Harrell',
 'Marc Coppey',
 'Maria Kliegel',
 'Mark Bernat',
 'Matt Haimovitz',
 'Matthieu Fontana',
 'Maurice Gendron',
 'Michael Goldschlag

One problem with this approach is that we find only the first artist associated with this recording. Most of the time however, this is not a problem since it is relatively rare for the database to have multiple artists associated with one recording.

In [8]:
get_recordings(12834183)

{'Alexandra Hawley, Robert Muczynski',
 'Laurel Ann Maurer, Joanne Pearce Martin',
 'Sanja Bizjak, Raquele Magalhães'}

Initially, when looking for the artists I joined the Artist table with the works table, but this gave some unexpected results: 
```
[(16895616,
  'Sonata, op. 14 for Flute and Piano: I. Allegro deciso',
  'Dirty Dubsters'),
 (19744636, 'Sonate pour flûte et piano, op. 14: I. Allegro decisio', None),
 (1953121,
  'Sonata for Flute and Piano, op. 14: I. Allegro deciso',
  'Robin Grean')]
```
I doubt the "Dirty Dubsters" recorded Muczynski's flute sonata. Later I found out there exists an artist_credit table. With this it is possible to get all artists as a field that can be split on ', '. However, it is possible for this field to contain aliases instead of the "official" name of the artists. In the ideal case I would first get the artists of an recording, then get the artist id of this artist alias, then get all the aliases for the artist. This would require some more work.

In [9]:
def get_artist(work_id, with_type=False):
    with conn:
        with conn.cursor() as curs:
            curs.execute(
                """
                select artist.name from work 
                left join l_artist_work on entity1=work.id 
                left join artist on entity0=artist.id 
                where work.id = %s;
                """, [work_id]
            )
        
            return curs.fetchall()[0][0]
            
get_artist(12834183)

'Robert Muczynski'